In [1]:
import os
import pandas as pd

docs = 'C:\\Users\\oyku_\\Desktop\\Recipe Collector\\Recipe-Collector\\pipeline\\data'
dataframes = []

for filename in os.listdir(docs):
    if filename.endswith('.csv'):
        file_path = os.path.join(docs, filename)
        
        try:
            df = pd.read_csv(file_path, delimiter=',', quotechar='"', on_bad_lines='skip')
            dataframes.append((filename, df))
        except Exception as e:
            print(f"Error reading {filename}: {e}")

print(f"Number of DataFrames loaded: {len(dataframes)}")
print("***********************************************************")
for filename, df in dataframes:
    print(f"{filename}: {df.shape[0]} rows")
print("***********************************************************")


for name, df in dataframes:
    print(f"Data from {name}:")
    print(df.head(3))
    print("\n")

combined_df = pd.concat([df for _, df in dataframes], ignore_index=True)

combined_csv_path ='C:\\Users\\oyku_\\Desktop\\Recipe Collector\\Recipe-Collector\\pipeline\\comb_rec.csv'
combined_df.to_csv(combined_csv_path, index=False)

print(f"Combined DataFrame saved to {combined_csv_path}")


Number of DataFrames loaded: 10
***********************************************************
bbc_food_recipes.csv: 24 rows
food_hero.csv: 500 rows
gordon_ramsay_recipes1.csv: 99 rows
Hello_Fresh.csv: 2347 rows
JamieOliverRecipes.csv: 631 rows
Morrisons.csv: 1283 rows
recipes_try.csv: 272 rows
Sainsburys.csv: 1713 rows
Tesco.csv: 739 rows
Treat_Kitchen.csv: 150 rows
***********************************************************
Data from bbc_food_recipes.csv:
                          Title  \
0  A return to the Black Forest   
1   Aberdeen butteries (Rowies)   
2            Ackee and saltfish   

                                         Ingredients  \
0  ['225g/8oz dark chocolate (75 per cent cocoa s...   
1  ['500g/1lb 2oz strong plain flour, plus extra ...   
2  ['1 tbsp vegetable oil', '2 onions, thinly sli...   

                                        Instructions  
0  ['Pre-heat the oven to 180C/350F/Gas 4.', "Bre...  
1  ['In a large bowl, mix together the flour, yea...  
2  ['For t

In [2]:
combined_csv_path = 'C:\\Users\\oyku_\\Desktop\\Recipe Collector\\Recipe-Collector\\pipeline\\comb_rec.csv'
combined_df = pd.read_csv(combined_csv_path)

print(combined_df.head(3))

                          Title  \
0  A return to the Black Forest   
1   Aberdeen butteries (Rowies)   
2            Ackee and saltfish   

                                         Ingredients  \
0  ['225g/8oz dark chocolate (75 per cent cocoa s...   
1  ['500g/1lb 2oz strong plain flour, plus extra ...   
2  ['1 tbsp vegetable oil', '2 onions, thinly sli...   

                                        Instructions  \
0  ['Pre-heat the oven to 180C/350F/Gas 4.', "Bre...   
1  ['In a large bowl, mix together the flour, yea...   
2  ['For the roasted peppers and onions (if using...   

  Title;Ingredients;Instructions  
0                            NaN  
1                            NaN  
2                            NaN  


In [6]:
sample_fraction = 0.1 #10% of the csv file

df_sampled = combined_df.sample(frac=sample_fraction, random_state=42)

print(df_sampled.head())

                                                  Title  \
2157     Roasted Chipotle Cauliflower and Chorizo Tacos   
3704                                      Spiced Mojito   
1323                                      Pasta Recipes   
6683  Chicken and pancetta salad with raspberries an...   
5328                      Emerald dhal with golden tofu   

                                            Ingredients  \
2157  300 grams, Cauliflower Florets, 1 unit(s), Lim...   
3704  Register, 1 handful of ice, 2 dashes of angost...   
1323                                                NaN   
6683  1 x 105g pack cooking smoked pancetta slices, ...   
5328  200g dried Puy lentils, rinsed, 2-2½ tbsp vege...   

                                           Instructions  \
2157  a) Preheat your oven to 220°C/200°C fan/gas ma...   
3704  Step 1Muddle lime wedges with mint and sugar s...   
1323                                                NaN   
6683  Slice the pancetta into strips and dry-fry for..

In [7]:
df_sampled

,Title,Ingredients,Instructions,Title;Ingredients;Instructions
2157,Roasted Chipotle Cauliflower and Chorizo Tacos,"300 grams, Cauliflower Florets, 1 unit(s), Lim...",a) Preheat your oven to 220°C/200°C fan/gas ma...,NaN
3704,Spiced Mojito,"Register, 1 handful of ice, 2 dashes of angost...",Step 1Muddle lime wedges with mint and sugar s...,NaN
1323,Pasta Recipes,NaN,NaN,NaN
6683,Chicken and pancetta salad with raspberries an...,"1 x 105g pack cooking smoked pancetta slices, ...",Slice the pancetta into strips and dry-fry for...,NaN
5328,Emerald dhal with golden tofu,"200g dried Puy lentils, rinsed, 2-2½ tbsp vege...",Tip the rinsed lentils into a saucepan and cov...,NaN
...,...,...,...,...
2280,Butter Paneer Baked Naan,"1 unit(s), Garlic Clove, 226 grams, Paneer, 1 ...",a) Preheat your oven to 220°C/200°C fan/gas ma...,NaN
4994,NaN,NaN,NaN,put the biscuits in a baking tin;
7197,Moroccan-style salmon with lemony couscous re...,"1tbsp olive oil, ½ onion, sliced, 1 garlic clo...",NaN,NaN
4348,Four Ingredient Avocado Pasta Sauce with Fibre...,"Register, 1 handful of basil leaves, 2 packs o...","Step 1Bring a large pan of water to the boil, ...",NaN
